# Preprocesamiento de Datos

Se hace una distribución de 70% para train (con validación cruzada) y 30% para test

In [102]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle
import os
import warnings

warnings.filterwarnings('ignore')

print("Librerías importadas correctamente")


Librerías importadas correctamente


In [103]:
# Cargar el dataset
df = pd.read_csv('ObesityDataSet_raw_and_data_sinthetic.csv')

print(f" Dataset cargado exitosamente")
print(f"  - Forma del dataset: {df.shape}")
print(f"  - Columnas: {list(df.columns)}")
print(f"\nPrimeras 3 filas:")
df.head(3)


 Dataset cargado exitosamente
  - Forma del dataset: (2111, 17)
  - Columnas: ['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight', 'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE', 'CALC', 'MTRANS', 'NObeyesdad']

Primeras 3 filas:


,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight


In [104]:
# Verificar valores faltantes
print(f"Valores faltantes por columna:")
missing = df.isnull().sum()
print(missing[missing > 0] if missing.sum() > 0 else "No hay valores faltantes")


Valores faltantes por columna:
No hay valores faltantes


Calculamos el IMC (BMI en inglés) a partir de Weight y Height, y se eliminan estas últimas dos 


IMC = Peso (kg) / Altura (m)²

Las clases de obesidad están basadas en rangos de IMC:
- Insufficient_Weight: IMC < 18.5
- Normal_Weight: 18.5 ≤ IMC < 25
- Overweight_Level_I: 25 ≤ IMC < 27
- Overweight_Level_II: 27 ≤ IMC < 30
- Obesity_Type_I: 30 ≤ IMC < 35
- Obesity_Type_II: 35 ≤ IMC < 40
- Obesity_Type_III: IMC ≥ 40


In [105]:
# Calcular IMC (BMI) = Weight (kg) / Height (m)²
print("Calculando IMC (Índice de Masa Corporal)...")
df['BMI'] = df['Weight'] / (df['Height'] ** 2)

print(f" IMC calculado")
print(f"  - Rango de IMC: {df['BMI'].min():.2f} - {df['BMI'].max():.2f}")
print(f"  - Media de IMC: {df['BMI'].mean():.2f}")

# Mostrar algunos ejemplos antes de eliminar
print(f"\nEjemplos de cálculo:")
print(df[['Height', 'Weight', 'BMI', 'NObeyesdad']].head())

# Eliminar Weight y Height (ya no las necesitamos)
print(f"\nEliminando variables Weight y Height...")
df = df.drop(columns=['Weight', 'Height'])
print(f"   Variables eliminadas")
print(f"  - Columnas restantes: {len(df.columns)}")

# Variable objetivo
target_column = 'NObeyesdad'
y = df[target_column].copy()

# Características (todas las columnas excepto la objetivo)
X = df.drop(columns=[target_column])

print(f"\n Variable objetivo separada: '{target_column}'")
print(f"  - Forma de y: {y.shape}")
print(f"  - Forma de X: {X.shape}")

# Identificar tipos de variables
print(f"\nANÁLISIS DE TIPOS DE VARIABLES:")
print("-" * 80)

# Variables numéricas (ya son números)
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
print(f"\nVariables Numéricas ({len(numeric_cols)}):")
for col in numeric_cols:
    print(f"  - {col}: min={X[col].min():.2f}, max={X[col].max():.2f}, media={X[col].mean():.2f}")

# Variables categóricas (objetos/strings)
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
print(f"\nVariables Categóricas ({len(categorical_cols)}):")
for col in categorical_cols:
    unique_vals = X[col].unique()
    print(f"  - {col}: {len(unique_vals)} valores únicos -> {list(unique_vals)[:5]}...")


Calculando IMC (Índice de Masa Corporal)...
 IMC calculado
  - Rango de IMC: 13.00 - 50.81
  - Media de IMC: 29.70

Ejemplos de cálculo:
   Height  Weight        BMI           NObeyesdad
0    1.62    64.0  24.386526        Normal_Weight
1    1.52    56.0  24.238227        Normal_Weight
2    1.80    77.0  23.765432        Normal_Weight
3    1.80    87.0  26.851852   Overweight_Level_I
4    1.78    89.8  28.342381  Overweight_Level_II

Eliminando variables Weight y Height...
   Variables eliminadas
  - Columnas restantes: 16

 Variable objetivo separada: 'NObeyesdad'
  - Forma de y: (2111,)
  - Forma de X: (2111, 15)

ANÁLISIS DE TIPOS DE VARIABLES:
--------------------------------------------------------------------------------

Variables Numéricas (7):
  - Age: min=14.00, max=61.00, media=24.31
  - FCVC: min=1.00, max=3.00, media=2.42
  - NCP: min=1.00, max=4.00, media=2.69
  - CH2O: min=1.00, max=3.00, media=2.01
  - FAF: min=0.00, max=3.00, media=1.01
  - TUE: min=0.00, max=2.00, med

### Encoding de variables categóricas

Label Encoding para variables binarias, One-Hot Encoding para variables categóricas

In [106]:
# Crear una copia para trabajar
X_encoded = X.copy()

# Identificar variables binarias (solo 2 valores)
binary_cols = []
multi_cat_cols = []

for col in categorical_cols:
    n_unique = X[col].nunique()
    if n_unique == 2:
        binary_cols.append(col)
    else:
        multi_cat_cols.append(col)

print(f"Variables Binarias (Label Encoding): {binary_cols}")
print(f"Variables Multi-Categoría (One-Hot Encoding): {multi_cat_cols}")


Variables Binarias (Label Encoding): ['Gender', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC']
Variables Multi-Categoría (One-Hot Encoding): ['CAEC', 'CALC', 'MTRANS']


In [107]:
# 4.1: Label Encoding para variables binarias
print(f"\nAplicando Label Encoding a variables binarias...")
label_encoders = {}

for col in binary_cols:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X[col])
    label_encoders[col] = le
    print(f"   {col}: {dict(zip(le.classes_, le.transform(le.classes_)))}")



Aplicando Label Encoding a variables binarias...
   Gender: {'Female': 0, 'Male': 1}
   family_history_with_overweight: {'no': 0, 'yes': 1}
   FAVC: {'no': 0, 'yes': 1}
   SMOKE: {'no': 0, 'yes': 1}
   SCC: {'no': 0, 'yes': 1}


In [108]:
# 4.2: One-Hot Encoding para variables multi-categoría
print(f"\nAplicando One-Hot Encoding a variables multi-categoría...")

# Usar pandas get_dummies (más simple que sklearn para este caso)
X_encoded = pd.get_dummies(X_encoded, columns=multi_cat_cols, prefix=multi_cat_cols, drop_first=False)

print(f"   Variables codificadas:")
print(f"    - Antes: {len(categorical_cols)} variables categóricas")
print(f"    - Después: {X_encoded.shape[1]} variables totales")
print(f"    - Nuevas columnas creadas: {X_encoded.shape[1] - len(numeric_cols) - len(binary_cols)}")

# Mostrar algunas columnas nuevas
new_cols = [col for col in X_encoded.columns if any(mc in col for mc in multi_cat_cols)]
print(f"\n  Ejemplo de nuevas columnas (primeras 5):")
for col in new_cols[:5]:
    print(f"    - {col}")



Aplicando One-Hot Encoding a variables multi-categoría...
   Variables codificadas:
    - Antes: 8 variables categóricas
    - Después: 25 variables totales
    - Nuevas columnas creadas: 13

  Ejemplo de nuevas columnas (primeras 5):
    - CAEC_Always
    - CAEC_Frequently
    - CAEC_Sometimes
    - CAEC_no
    - CALC_Always


### Normalización/Estandarización de Variables Numéricas

Estandarización (Z-score normalization)


In [109]:
# Identificar columnas numéricas (después del encoding)
# Las columnas numéricas originales + las binarias codificadas
numeric_cols_to_scale = numeric_cols + binary_cols

print(f"Variables a estandarizar ({len(numeric_cols_to_scale)}):")
print(f"  {numeric_cols_to_scale}")

# Crear el scaler (pero NO aplicarlo aún - lo haremos después de dividir)
# Esto es importante: NO debemos estandarizar antes de dividir train/test
# porque podríamos "filtrar" información del test al train

scaler = StandardScaler()

print(f"\n Scaler creado (se aplicará después de dividir train/test)")
print(f"  - Tipo: StandardScaler (Z-score normalization)")
print(f"  - Fórmula: z = (x - μ) / σ")

# Mostrar estadísticas antes de estandarizar
print(f"\n Estadísticas ANTES de estandarizar (primeras 3 variables):")
for col in numeric_cols_to_scale[:3]:
    print(f"  {col}:")
    print(f"    - Media: {X_encoded[col].mean():.4f}")
    print(f"    - Desv. Est.: {X_encoded[col].std():.4f}")
    print(f"    - Min: {X_encoded[col].min():.4f}")
    print(f"    - Max: {X_encoded[col].max():.4f}")


Variables a estandarizar (12):
  ['Age', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE', 'BMI', 'Gender', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC']

 Scaler creado (se aplicará después de dividir train/test)
  - Tipo: StandardScaler (Z-score normalization)
  - Fórmula: z = (x - μ) / σ

 Estadísticas ANTES de estandarizar (primeras 3 variables):
  Age:
    - Media: 24.3126
    - Desv. Est.: 6.3460
    - Min: 14.0000
    - Max: 61.0000
  FCVC:
    - Media: 2.4190
    - Desv. Est.: 0.5339
    - Min: 1.0000
    - Max: 3.0000
  NCP:
    - Media: 2.6856
    - Desv. Est.: 0.7780
    - Min: 1.0000
    - Max: 4.0000


### División de Datos (70-30) con Estratificación

Divide los datos en Train (70%) y Test (30%) manteniendo la proporción de clases en ambos conjuntos.


In [110]:
# División estratificada
# random_state: Para reproducibilidad (mismos resultados cada vez)
# stratify: Mantiene proporción de clases
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded,
    y,
    test_size=0.30,  # 30% para test
    random_state=42,  # Semilla para reproducibilidad
    stratify=y  # Estratificación por clases
)

print(f"\n División completada:")
print(f"  - Train: {X_train.shape[0]} registros ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"  - Test: {X_test.shape[0]} registros ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"  - Características: {X_train.shape[1]}")



 División completada:
  - Train: 1477 registros (70.0%)
  - Test: 634 registros (30.0%)
  - Características: 25


In [111]:
# Verificar estratificación
print(f"\n Verificación de estratificación:")
print(f"\nDistribución en Train:")
train_dist = y_train.value_counts().sort_index()
train_pct = (y_train.value_counts(normalize=True) * 100).sort_index()
for clase in train_dist.index:
    print(f"  {clase:25s}: {train_dist[clase]:4d} ({train_pct[clase]:5.2f}%)")

print(f"\nDistribución en Test:")
test_dist = y_test.value_counts().sort_index()
test_pct = (y_test.value_counts(normalize=True) * 100).sort_index()
for clase in test_dist.index:
    print(f"  {clase:25s}: {test_dist[clase]:4d} ({test_pct[clase]:5.2f}%)")

# Verificar que las proporciones son similares
print(f"\n Verificación: Las proporciones son similares en ambos conjuntos")



 Verificación de estratificación:

Distribución en Train:
  Insufficient_Weight      :  190 (12.86%)
  Normal_Weight            :  201 (13.61%)
  Obesity_Type_I           :  245 (16.59%)
  Obesity_Type_II          :  208 (14.08%)
  Obesity_Type_III         :  227 (15.37%)
  Overweight_Level_I       :  203 (13.74%)
  Overweight_Level_II      :  203 (13.74%)

Distribución en Test:
  Insufficient_Weight      :   82 (12.93%)
  Normal_Weight            :   86 (13.56%)
  Obesity_Type_I           :  106 (16.72%)
  Obesity_Type_II          :   89 (14.04%)
  Obesity_Type_III         :   97 (15.30%)
  Overweight_Level_I       :   87 (13.72%)
  Overweight_Level_II      :   87 (13.72%)

 Verificación: Las proporciones son similares en ambos conjuntos


### Estandarización de datos después de dividir para evitar leakage



In [112]:
# Ajustar scaler SOLO con datos de train
print(f"\n Ajustando scaler con datos de train...")
scaler.fit(X_train[numeric_cols_to_scale])

# Aplicar transformación
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[numeric_cols_to_scale] = scaler.transform(X_train[numeric_cols_to_scale])
X_test_scaled[numeric_cols_to_scale] = scaler.transform(X_test[numeric_cols_to_scale])

print(f" Estandarización aplicada")

# Mostrar estadísticas después de estandarizar
print(f"\n Estadísticas DESPUÉS de estandarizar (primeras 3 variables):")
for col in numeric_cols_to_scale[:3]:
    print(f"  {col}:")
    print(f"    - Media: {X_train_scaled[col].mean():.6f} (debe ser ~0)")
    print(f"    - Desv. Est.: {X_train_scaled[col].std():.6f} (debe ser ~1)")
    print(f"    - Min: {X_train_scaled[col].min():.4f}")
    print(f"    - Max: {X_train_scaled[col].max():.4f}")



 Ajustando scaler con datos de train...
 Estandarización aplicada

 Estadísticas DESPUÉS de estandarizar (primeras 3 variables):
  Age:
    - Media: 0.000000 (debe ser ~0)
    - Desv. Est.: 1.000339 (debe ser ~1)
    - Min: -1.3202
    - Max: 5.8305
  FCVC:
    - Media: -0.000000 (debe ser ~0)
    - Desv. Est.: 1.000339 (debe ser ~1)
    - Min: -2.7065
    - Max: 1.0833
  NCP:
    - Media: -0.000000 (debe ser ~0)
    - Desv. Est.: 1.000339 (debe ser ~1)
    - Min: -2.1733
    - Max: 1.6959


### Encoding variable objetivo

```
0: Insufficient_Weight
1: Normal_Weight
2: Overweight_Level_I
3: Overweight_Level_II
4: Obesity_Type_I
5: Obesity_Type_II
6: Obesity_Type_III
```

In [113]:
# Definir orden ordinal
ordinal_order = [
    'Insufficient_Weight',
    'Normal_Weight',
    'Overweight_Level_I',
    'Overweight_Level_II',
    'Obesity_Type_I',
    'Obesity_Type_II',
    'Obesity_Type_III'
]

# Crear encoder ordinal
target_encoder = LabelEncoder()
target_encoder.fit(ordinal_order)

# Aplicar encoding
y_train_encoded = pd.Series(target_encoder.transform(y_train), index=y_train.index)
y_test_encoded = pd.Series(target_encoder.transform(y_test), index=y_test.index)

print(f"\n Variable objetivo codificada:")
print(f"  Mapeo de clases:")
for i, clase in enumerate(ordinal_order):
    print(f"    {i}: {clase}")

print(f"\n  Distribución en Train (codificada):")
print(y_train_encoded.value_counts().sort_index())

print(f"\n  Distribución en Test (codificada):")
print(y_test_encoded.value_counts().sort_index())



 Variable objetivo codificada:
  Mapeo de clases:
    0: Insufficient_Weight
    1: Normal_Weight
    2: Overweight_Level_I
    3: Overweight_Level_II
    4: Obesity_Type_I
    5: Obesity_Type_II
    6: Obesity_Type_III

  Distribución en Train (codificada):
0    190
1    201
2    245
3    208
4    227
5    203
6    203
Name: count, dtype: int64

  Distribución en Test (codificada):
0     82
1     86
2    106
3     89
4     97
5     87
6     87
Name: count, dtype: int64


### Guardado de datos preprocesados

In [114]:
# Crear directorio si no existe
os.makedirs('data/processed', exist_ok=True)
os.makedirs('models/preprocessing', exist_ok=True)

# Guardar datos preprocesados
print(f"\n Guardando datos preprocesados...")

# Guardar como CSV (para inspección)
X_train_scaled.to_csv('data/processed/X_train.csv', index=False)
X_test_scaled.to_csv('data/processed/X_test.csv', index=False)
y_train_encoded.to_csv('data/processed/y_train.csv', index=False)
y_test_encoded.to_csv('data/processed/y_test.csv', index=False)

# Guardar también las versiones originales (sin codificar) para referencia
y_train.to_csv('data/processed/y_train_original.csv', index=False)
y_test.to_csv('data/processed/y_test_original.csv', index=False)

print(f"   Datos guardados en 'data/processed/'")



 Guardando datos preprocesados...
   Datos guardados en 'data/processed/'


In [115]:
# Guardar transformadores
print(f"\n Guardando transformadores...")

with open('models/preprocessing/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

with open('models/preprocessing/target_encoder.pkl', 'wb') as f:
    pickle.dump(target_encoder, f)

with open('models/preprocessing/label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)

# Guardar información sobre las columnas
preprocessing_info = {
    'numeric_cols': numeric_cols,
    'binary_cols': binary_cols,
    'multi_cat_cols': multi_cat_cols,
    'numeric_cols_to_scale': numeric_cols_to_scale,
    'ordinal_order': ordinal_order,
    'n_features': X_train_scaled.shape[1]
}

with open('models/preprocessing/preprocessing_info.pkl', 'wb') as f:
    pickle.dump(preprocessing_info, f)

print(f"   Transformadores guardados en 'models/preprocessing/'")



 Guardando transformadores...
   Transformadores guardados en 'models/preprocessing/'


### Resumen

1. Carga y exploración del dataset original
2. Identificación de tipos de variables (numéricas vs categóricas)
3. Encoding de variables
4. Estandarización de variables numéricas
5. División estratificada** 70-30 (Train/Test)
6. Encoding ordinal** de variable objetivo
7. Guardado de datos preprocesados y transformadores
